# Cuaderno 26: Modelo para el TSP con desigualdades de corte

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\RR}{\mathbb{R}}$

Dados: 
* un grafo no dirigido **completo** $G=(V,E)$; y,
* un vector $c \in \ZZ^{E}$ de costos asociados a las aristas de $G$.

El *problema del agente viajero (Traveling Salesman Problem, TSP)* consiste en encontrar un ciclo que visite **todos** los nodos de $G$ y que tenga el menor costo posible.

Al tratarse de una versión no dirigida del problema, las restricciones de grado entrante y saliente se reemplazan por una única restricción de grado para cada nodo: cada nodo debe ser incidente a dos aristas del tour.

Por otra parte, notar que para impedir subciclos en una solución es suficiente con requerir que la misma sea conexa,
lo cual puede hacerse a través de desigualdades de corte, similares al caso del árbol generador de peso mínimo. En el caso del TSP, estas desigualdades pueden ser mejoradas al observar que, para cualquier conjunto $W \subset V$ con $\emptyset \neq W \neq V$, todo tour factible debe contener al menos *dos* aristas del corte $\delta(W)$.

Utilizando variables binarias $x_{ij}$ para indicar la selección de aristas, el problema del agente viajero puede formularse como el siguiente programa lineal entero:

\begin{align*}
\min &\sum_{ij \in E} c_{ij} x_{ij}\\ 
& \mbox{s.r.}\\
&\sum_{ij \in \delta(i) } x_{ij} = 2, \quad \forall i \in V,\\
&\sum_{ij \in \delta(W)} x_{ij} \geq 2, \quad \forall W \subset V, \emptyset \neq W \neq V,\\
& x_{ij} \in \{0, 1\}, \quad \forall ij \in E,
\end{align*}
donde $\delta(W) := \{ ij \in E \, : \, i \in W, j \not\in W\}$.

La función objetivo mide el costo total de las aristas seleccionadas.

La primera familia de restricciones contiene las restricciones de grado para cada nodo.

La segunda familia de restricciones contiene un número exponencial de restricciones para la eliminación de subciclos, empleando la observación señalada arriba.

Vamos a implementar este modelo usando la interfaz Python de Gurobi.



Definimos primero los datos. Usaremos la función `randint` del módulo random para generar valores aleatorios en el rango {0,..,100} para las coordenadas de cada nodo. Los costos de las aristas serán iguales a las distancias euclideanas entre sus nodos extremos. Notar que el diccionario `c` contiene cada arista una sola vez, debido a la condición `if i < j`.

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import random as rm
import math 

# Número de nodos del grafo
n = 10

# Nodos del grafo
V = gp.tuplelist(range(1,n+1))

# Posiciones de los nodos en un plano euclideano entre (0,0) y (100, 100)
coordx={i : rm.randint(0,100) for i in V}
coordy={i : rm.randint(0,100) for i in V}

# los costos son las distancias eculideanas
c = gp.tupledict({(i,j) : math.sqrt((coordx[i] - coordx[j])**2 + (coordy[i] - coordy[j])**2)
              for i in V for j in V if i < j})
 
E = c.keys()


Vamos a necesitar construir una restricción de corte para cada subconjunto de nodos $W \subset V$ tal que $\emptyset \neq W \neq V$. Con esta finalidad, definimos una función `powerset` empleando las funciones `chain` y `combinations` del módulo `itertools`.

In [ ]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

# Ejemplo
for W in powerset([1,2,3]):
    print(W)

Emplearemos la función `dibujarTour` para graficar el tour utilizando el módulo `matplotlib`. Esta función recibe tres argumentos: una lista `coordx` con las coordenadas horizontales de los nodos, una lista `coordy` con las coordenadas verticales y un vector `tour` con una permutación de los nodos indicando el orden de visita en la solución.

In [ ]:
import matplotlib.pyplot as plt
def dibujarTour(coordx, coordy, tour):
    Tx = [coordx[i] for i in tour] + [coordx[i] for i in tour[:1]]
    Ty = [coordy[i] for i in tour] + [coordy[i] for i in tour[:1]]
    plt.plot(Tx, Ty, 'b-')
    plt.plot(Tx[:-1], Ty[:-1], 'ro')
    for i in range(len(tour)-1):
        s='{}'.format(tour[i])
        plt.text(Tx[i],Ty[i]+1,s)
    plt.show()


Definimos ahora el objeto modelo, las variables y la función objetivo:

In [ ]:
# Crear el objeto modelo
m = gp.Model('tsp-corte')

# Crear las variables
x = m.addVars(E, name="x", vtype=GRB.BINARY)

# Crear la función objetivo
m.setObjective(x.prod(c,'*'), GRB.MINIMIZE)


Añadimos las restricciones de grado. Observar que son necesarias **dos sumas** en cada restricción para cubrir todas las aristas del corte $\delta(i)$: 

In [ ]:
# Restricciones de grado
m.addConstrs((x.sum(i,'*') + x.sum('*', i) == 2 for i in V), 
                 "grado")

Para construir las restricciones de corte, usamos la función `powerset` para iterar sobre todos los subconjuntos de nodos $W$ tales que $\emptyset \neq W \neq V$. Para cada subconjunto, se construye primero el conjunto `Wc` con su 
complemento, es decir, con los nodos de $V$ que no están en $W$. La expresión `x.sum(W, Wc)` construye la suma de las variables asociadas a aristas que tienen el "primer" extremo en $W$ y el otro extremo fuera de $W$. De manera similar, `x.sum(Wc, W)` construye la suma de las variables asociadas a aristas que tienen el "primer" extremo fuera de $W$ y el otro extremo en $W$. La suma de ambas expresiones es igual a la suma de todas las aristas del corte.

In [ ]:
# Restricciones de corte
for W in powerset(V):
    if W!=() and len(W)!=len(V):
        Wc = [i for i in V if i not in W]
        m.addConstr(x.sum(W, Wc) + x.sum(Wc, W) >= 2, 
                        "corte[{}]".format(W))


Escribimos el modelo a un archivo de texto:

In [ ]:
# Escribir el modelo a un archivo
m.write('tsp-corte.lp')


Finalmente, resolvemos el modelo y mostramos la solución:

In [ ]:
# Calcular la solución óptima
m.optimize()

# Escribir la solución
if m.SolCount > 0:
    # Recuperar los valores de las variables
    vx = m.getAttr('x', x)
    print('\nTour óptimo:')
    for i,j in E:
        if vx[i,j] >= 0.9:
            print('{} -- {}'.format(i, j))


Finalmente, graficamos la solución obtenida:

In [ ]:
# Crear lista con arcos seleccionados en la solución
L = gp.tuplelist([(i,j) for i,j in E if vx[i,j]>0])

# Recuperar el tour como un ordenamiento de los nodos
T = [1]
# nodo actual:
i = 1
while True:
    # Determinar sucesor de i en el tour
    # Observar que la arista puede ser de la forma (i,j) o (j,i)
    N = L.select(i,'*') +  L.select('*',i)
    a = N.pop()
    L.remove(a)
    j = a[1] if i==a[0] else a[0]
    # Colocar sucesor en la lista del tour y actualizar i
    T.append(j)
    i = j
    # Terminar cuando el nodo colocado sea 1
    if i==1: 
        break;
        
# Graficar el tour
dibujarTour(coordx, coordy, T)

## Código completo

Se reproduce a continuación el código completo del modelo anterior.

In [ ]:
# Implementación de modelos lineales enteros
# Problema del agente viajero simétrico (TSP)

# Luis M. Torres (EPN 2022)

import gurobipy as gp
from gurobipy import GRB
import random as rm
import matplotlib.pyplot as plt
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

def dibujarTour(coordx, coordy, tour):
    Tx = [coordx[i] for i in tour] + [coordx[i] for i in tour[:1]]
    Ty = [coordy[i] for i in tour] + [coordy[i] for i in tour[:1]]
    plt.plot(Tx, Ty, 'b-')
    plt.plot(Tx[:-1], Ty[:-1], 'ro')
    for i in range(len(tour)-1):
        s='{}'.format(tour[i])
        plt.text(Tx[i],Ty[i]+1,s)
    display(plt.show())

# Número de nodos del grafo
n = 10

# Nodos del grafo
V = gp.tuplelist(range(1,n+1))

# Posiciones de los nodos en un plano euclideano entre (0,0) y (100, 100)
coordx={i : rm.randint(0,100) for i in V}
coordy={i : rm.randint(0,100) for i in V}

# los costos son las distancias eculideanas
c = gp.tupledict({(i,j) : math.sqrt((coordx[i] - coordx[j])**2 + (coordy[i] - coordy[j])**2)
              for i in V for j in V if i < j})
 
E = c.keys()

try:
    # Crear el objeto modelo
    m = gp.Model('tsp-corte')

    # Crear las variables
    x = m.addVars(E, name="x", vtype=GRB.BINARY)

    # Crear la función objetivo
    m.setObjective(x.prod(c,'*'), GRB.MINIMIZE)

    # Restricciones de grado
    m.addConstrs((x.sum(i,'*') + x.sum('*', i) == 2 for i in V), 
                 "grado")

    # Restricciones de corte
    PV = list(powerset(V))
    for W in PV:
        if W!=() and len(W)!=len(V):
            Wc = [i for i in V if i not in W]
            m.addConstr(x.sum(W, Wc) + x.sum(Wc, W) >= 1, 
                            "corte[{}]".format(W))

    # Escribir el modelo a un archivo
    # m.write('tsp-corte.lp')

    # Calcular la solución óptima
    m.optimize()

    # Escribir la solución
    if m.SolCount > 0:
        # Recuperar los valores de las variables
        vx = m.getAttr('x', x)
        print('\nTour óptimo:')
        for i,j in E:
            if vx[i,j] >= 0.9:
                print('{} -> {}'.format(i, j))
                
    # Crear lista con arcos seleccionados en la solución
    L = gp.tuplelist([(i,j) for i,j in E if vx[i,j]>0])

    # Recuperar el tour como un ordenamiento de los nodos
    T = [1]
    # nodo actual:
    i = 1
    while True:
        # Determinar sucesor de i en el tour
        # Observar que la arista puede ser de la forma (i,j) o (j,i)
        N = L.select(i,'*') +  L.select('*',i)
        a = N.pop()
        L.remove(a)
        j = a[1] if i==a[0] else a[0]
        # Colocar sucesor en la lista del tour y actualizar i
        T.append(j)
        i = j
        # Terminar cuando el nodo colocado sea 1
        if i==1: 
            break;
        
    # Graficar el tour
    dibujarTour(coordx, coordy, T)    
    
except GurobiError as e:
    print('Se produjo un error de Gurobi: código: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')